# Train macro & fundamental-aware price models 
Pretraining with fundamental, macroeconomic, estimate and sharep price data to capture the data patterns.
Use embedded fundamental/macro/short-term information for return prediction


## 1. Load in data

In [1]:
from data import FundamentalDataset, PriceDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import datetime as dt
import itertools
from utils import Defaults
from torch.utils.data import DataLoader, Dataset
from copy import deepcopy

DEFAULTS = Defaults
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


fund_data = FundamentalDataset()
fund_data_weekly = FundamentalDataset(freq="W")
price_data = PriceDataset()

def collate_fn(batch):
    data_ls, masks = [], []
    for data, mask in batch:
        data_ls.append(data)
        masks.append(mask)
    return (
        torch.stack(data_ls),
        torch.stack(masks)
    )

## 2. Train autoencoders as pre-training

### 2.1. Train encoders on fundamental data

In [2]:
from typing import Sequence

def expand_mask(mask: torch.tensor, target_dim: int) -> torch.tensor:
    """expand mask from n dimensions to n+1 dimensions"""
    newmask = deepcopy(mask).unsqueeze(-1)
    mask_dims = list(newmask.shape)
    mask_dims[-1] = target_dim
    mask_dims = tuple(mask_dims)
    return newmask.expand(mask_dims)

def expand_masks(masks: Sequence[torch.tensor], target_dims: Sequence[int]):
    expanded_masks = []
    for mask, dim in zip(masks, target_dims):
        newmask = expand_mask(mask, dim)
        expanded_masks.append(newmask)
    return expanded_masks

In [3]:
def masked_mse_loss(
        input: torch.tensor, 
        target: torch.tensor,
        mask: torch.tensor,
        na_pad: torch.tensor,
        ) -> torch.tensor:
    """custome MSE loss to mask padding & nan values
    :param input: original vector
    :param target: target vector
    :param 
    """
    loss = nn.MSELoss()
    dims = input.shape[-1]
    na_mask = input == na_pad
    expanded_mask = expand_mask(mask, dims)
    new_mask = na_mask.astype(torch.bool) + expanded_mask.astype(torch.bool)
    masked_input = torch.masked_select(input, ~new_mask) # mask itself is True if masked
    masked_target = torch.masked_select(target, ~new_mask)
    return loss(masked_input, masked_target)

def composite_mseloss(mse_losses: Sequence[torch.tensor]):
    mean_loss = torch.mean(mse_losses)
    penalty_loss = torch.sum([(loss - mean_loss)**2 for loss in mse_losses])
    composite_loss = mean_loss + penalty_loss
    return composite_loss

def multiple_input_masked_mse_loss(
        inputs: Sequence[torch.tensor],
        targets: Sequence[torch.tensor],
        masks: Sequence[torch.tensor],
        na_pads: Sequence[torch.tensor]):
    losses = []
    for input, target, mask, na_pad in zip(
        inputs, targets, masks, na_pads):
        loss = masked_mse_loss(input, target, mask, na_pad)
        losses.append(loss)
    composite_loss = composite_mseloss(losses)
    return composite_loss


In [14]:
print(model.linear_encoder_layers[0])

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): BatchNorm1d(170, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Linear(in_features=170, out_features=42, bias=True)
  (3): Linear(in_features=42, out_features=10, bias=True)
  (4): Linear(in_features=10, out_features=2, bias=True)
  (5): Linear(in_features=2, out_features=3, bias=True)
)


In [4]:
from typing import Sequence, Tuple, Optional
def encode(
        model, 
        inputs: Sequence[Tuple[torch.tensor, Optional[torch.tensor]]],
        padding_masks: Sequence[torch.tensor]
        ) -> Tuple[torch.tensor, Tuple[torch.tensor]]:
    """encode a list of inputs of different lengths and dimensionalities
    into a single embedding vector
    """
    embeddings, memories = [], []
    for input, mask, transformer_encoder, linear_encoder in zip(
        inputs, padding_masks, model.transformer_encoders, model.linear_encoder_layers):
        print(input.shape)
        x_ = transformer_encoder(input, src_key_padding_mask=mask)
        memories.append(x_)
        embedded = linear_encoder(x_)
        print(embedded.shape)
        embeddings.append(embedded)
    _embedding = torch.stack(embeddings, dim=-1)
    print(_embedding.shape)
    embedding = model.linear_encoder(_embedding)
    print(embedding.shape)
    embedding = model.tanh(embedding)
    print("finished encoder\n===\n")
    return (embedding, memories)


def decode(
        model, 
        embedding: torch.tensor, 
        memories: Sequence[torch.tensor]) -> Sequence[torch.tensor]:
    print("decoder input shape: \n", embedding.shape)
    _embeddings = model.linear_decoder(embedding)
    print("after linear embedding shape:\n", _embeddings.shape)
    reconstructed_xs = []
    for i in range(model.num_inputs):
        _output = model.linear_decoder_layers[i](_embeddings[:, :, i])
        output = model.transformer_decoders[i](_output, memory=memories[i])
        reconstructed_xs.append(output)
    return reconstructed_xs

In [9]:
input, mask = input.to(DEVICE), mask.to(DEVICE)
embedding, memories = encode(model, [input], [mask])
output = decode(model, embedding, memories)

torch.Size([16, 10, 17])
torch.Size([16, 3])
torch.Size([16, 3, 1])
torch.Size([16, 3])
finished encoder
===

decoder input shape: 
 torch.Size([16, 3])
after linear embedding shape:
 torch.Size([16, 3, 1])


In [11]:
from models.autoencoder import BaseAutoEncoder
from torch.utils.tensorboard import SummaryWriter

RUN = 1
LR = 1e-4
NUM_TRANSFORMER_LAYERS = 5
WINDOW_SIZE = 10
NHEADS = 1
ENCODING_DIM = 3
MAX_EPOCHS = 10
BATCH_SIZE = 16

logger_stem = "./traininglog/fundamental_encoder/runs/"
logger = SummaryWriter(f"{logger_stem}run{RUN};lr={LR};notflayrs={NUM_TRANSFORMER_LAYERS};wd={WINDOW_SIZE};nh={NHEADS};edim={ENCODING_DIM};bsize={BATCH_SIZE}")

fundamental_data_loader = DataLoader(
    fund_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=BATCH_SIZE
)

num_batches = len(fundamental_data_loader)

model = BaseAutoEncoder(
    window_sizes=[WINDOW_SIZE],
    encoding_dim=ENCODING_DIM, 
    num_transformer_layers=[NUM_TRANSFORMER_LAYERS], 
    dims=[17],
    activation_func=F.tanh,
    nheads=[NHEADS],
    device=DEVICE)
model = model.to(DEVICE)

optimizer = torch.optim.Adam(
    model.parameters(), lr=LR, betas=[0.9, 0.99], eps=1e-07)



for epoch in range(MAX_EPOCHS):
    running_losses = []
    for i, (input, mask) in enumerate(fundamental_data_loader):
        # forward pass
        input, mask = input.to(DEVICE), mask.to(DEVICE)
        embedding, memories = model.encode([input], [mask])
        output = model.decode(embedding, memories)
        loss = masked_mse_loss(input, output, mask, na_pad=DEFAULTS.padding_val)
        running_losses.append(loss.item())
        logger.add_scaler("loss/train_step", loss.item(), step=epoch*num_batches + i)

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    mean_loss = np.mean(running_losses)
    logger.add_scalar("loss/train", mean_loss, step=epoch)


/anaconda/envs/stockencoder_env/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


torch.Size([16, 3, 1])
torch.Size([16, 3])


AttributeError: 'Tensor' object has no attribute 'astype'

In [ ]:
price_data[(1, dt.date(2022, 1, 1))]

c:\Users\p.peng\Anaconda3\envs\stockencoder_env\lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


tensor([[-1.0000e+10,  2.2513e-01, -1.0000e+10,  8.9167e-01,  9.5669e-01],
        [-1.0000e+10,  2.2591e-01, -1.0000e+10,  8.9041e-01,  9.5619e-01],
        [-1.0000e+10,  2.2721e-01, -1.0000e+10,  8.8854e-01,  9.5544e-01],
        ...,
        [-1.0000e+10,  2.7621e-01, -6.6862e+00,  8.8338e-01,  9.2140e-01],
        [-1.0000e+10,  2.7616e-01, -2.5919e+00,  8.8354e-01,  9.2150e-01],
        [-1.0000e+10,  2.7332e-01, -1.0000e+10,  8.9210e-01,  9.2715e-01]])